# Code for Classification of toric colorable seeds of Picard number $4$
In terms of:
    - Positivity
    - Fan-givingness
    - Projectivity

In [1]:
import SimplicialComplex as sc
import json
import sympy as sp
import numpy as np
from itertools import combinations,permutations

def read_file(filename):
    with open(filename, 'rb') as f:
        data = f.readlines()
        data = [x.strip() for x in data]
    return data

### Function for loading all the seeds of a given pair $(m,n)$

In [2]:
def load_seeds(n,m):
    db_path = 'final_results/CSPLS_%d_%d' % (n, m)
    list_facets = [json.loads(facets_bytes) for facets_bytes in read_file(db_path)]
    return [sc.PureSimplicialComplex(facets_set) for facets_set in list_facets]

    
    

### Sympy to Z3py

In [3]:
from z3 import Int, Sqrt 
from sympy.core import Mul, Expr, Add, Pow, Symbol, Number

def sympy_to_z3(sympy_var_list, sympy_exp):
    'convert a sympy expression to a z3 expression. This returns (z3_vars, z3_expression)'

    z3_vars = []
    z3_var_map = {}

    for var in sympy_var_list:
        name = var.name
        z3_var = Int(name)
        z3_var_map[name] = z3_var
        z3_vars.append(z3_var)

    result_exp = _sympy_to_z3_rec(z3_var_map, sympy_exp)

    return z3_vars, result_exp

def _sympy_to_z3_rec(var_map, e):
    'recursive call for sympy_to_z3()'

    rv = None

    if not isinstance(e, Expr):
        raise RuntimeError("Expected sympy Expr: " + repr(e))

    if isinstance(e, Symbol):
        rv = var_map.get(e.name)

        if rv == None:
            raise RuntimeError("No var was corresponds to symbol '" + str(e) + "'")

    elif isinstance(e, Number):
        rv = float(e)
    elif isinstance(e, Mul):
        rv = _sympy_to_z3_rec(var_map, e.args[0])

        for child in e.args[1:]:
            rv *= _sympy_to_z3_rec(var_map, child)
    elif isinstance(e, Add):
        rv = _sympy_to_z3_rec(var_map, e.args[0])

        for child in e.args[1:]:
            rv += _sympy_to_z3_rec(var_map, child)
    elif isinstance(e, Pow):
        term = _sympy_to_z3_rec(var_map, e.args[0])
        exponent = _sympy_to_z3_rec(var_map, e.args[1])

        if exponent == 0.5:
            # sqrt
            rv = Sqrt(term)
        else:
            rv = term**exponent

    if rv == None:
        raise RuntimeError("Type '" + str(type(e)) + "' is not yet implemented for convertion to a z3 expresion. " + \
                            "Subexpression was '" + str(e) + "'.")

    return rv

### Function for testing positivity of a pair $(K,\lambda^{\mathbb{R}})$

In [9]:
from z3 import Solver, sat

def is_positive(K:sc.PureSimplicialComplex,mod_2_IDCM,orientation):
    (m,n) = (K.m,K.n)
    mod_2_char_map = np.zeros((n,m))
    mod_2_char_map[:,:n] = np.eye(n)
    mod_2_char_map[:,n:] = mod_2_IDCM[:,:n].T
    x_vars = sp.symarray('x',(n,m-n))
    R = sp.ZZ.old_poly_ring(x_vars)
    poly_char_map = sp.Matrix(mod_2_char_map)
    for i in range(n):
        for j in range(m):
            if mod_2_char_map[i,j]==1:
                poly_char_map[i,j] = sp.Integer(1)
            else:
                poly_char_map[i,j] = sp.Integer(0)
    poly_char_map[:,n:] += sp.Integer(2)*x_vars
    N = len(K.facets_bin)
    s = Solver()
    for k in range(1,len(K.facets_bin)):
        z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],poly_char_map[:,sc.binary_to_face_0(K.facets_bin[k],m)].det().expand()-sp.Integer(orientation[k]))
        s.add(z3_poly==0)
    # for i in range(N):
    #     for j in range(i+1,N):
    #         intersection = sc.binary_to_face_0(K.facets_bin[i]&K.facets_bin[j],m)
    #         if len(intersection)==n-1:
    #             vertices = sc.binary_to_face_0(K.facets_bin[i]^K.facets_bin[j],m)
    #             intersection1 = intersection.copy()
    #             intersection1.append(vertices[0])
    #             intersection2 = intersection.copy()
    #             intersection2.append(vertices[1])
    #             z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],(poly_char_map[:,intersection1].det()*poly_char_map[:,intersection2].det()).expand()+sp.Integer(1)) 
    #             s.add(z3_poly==0)
    test = s.check()
    return test==sat

def is_positive_and_pm1(K:sc.PureSimplicialComplex,mod_2_IDCM,orientation):
    (m,n) = (K.m,K.n)
    mod_2_char_map = np.zeros((n,m))
    mod_2_char_map[:,:n] = np.eye(n)
    mod_2_char_map[:,n:] = mod_2_IDCM[:,:n].T
    x_vars = sp.symarray('x',(n,m-n))
    R = sp.ZZ.old_poly_ring(x_vars)
    poly_char_map = sp.Matrix(mod_2_char_map)
    for i in range(n):
        for j in range(m):
            if mod_2_char_map[i,j]==1:
                poly_char_map[i,j] = sp.Integer(1)
            else:
                poly_char_map[i,j] = sp.Integer(0)
    poly_char_map[:,n:] -= sp.Integer(2)*x_vars
    N = len(K.facets_bin)
    s = Solver()
    for k in range(n):
        for l in range(m-n):
            z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],x_vars[k,l]*(x_vars[k,l]-1))
            s.add(z3_poly==0)
    for k in range(1,N):
        z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],poly_char_map[:,sc.binary_to_face_0(K.facets_bin[k],m)].det().expand()-sp.Integer(orientation[k]))
        s.add(z3_poly==0)
    # for i in range(N):
    #     for j in range(i+1,N):
    #         intersection = sc.binary_to_face_0(K.facets_bin[i]&K.facets_bin[j],m)
    #         if len(intersection)==n-1:
    #             vertices = sc.binary_to_face_0(K.facets_bin[i]^K.facets_bin[j],m)
    #             intersection1 = intersection.copy()
    #             intersection1.append(vertices[0])
    #             intersection2 = intersection.copy()
    #             intersection2.append(vertices[1])
    #             z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],(poly_char_map[:,intersection1].det()*poly_char_map[:,intersection2].det()).expand()+sp.Integer(1)) 
    #             s.add(z3_poly==0)
    test = s.check()
    return test==sat


In [7]:
import tqdm


index_positive_5_9=[6, 7, 8, 10,  11, 18, 19, 20, 28, 47, 53, 76, 92, 95, 97, 108]
Data_5_9 = load_seeds(5,9)
index_non_positive_5_9=[k for k in range(len(Data_5_9)) if k not in index_positive_5_9]

index_positive_6_10=[5,6,11,55,72,126,127,188,271,293,309,326,330,352,392,550,583,616]
Data_6_10 = load_seeds(6,10)
index_non_positive_6_10=[k for k in range(len(Data_6_10)) if k not in index_positive_6_10]


index_positive_7_11= [12, 162, 210, 247, 355, 359, 556, 1117, 1155]
Data_7_11 = load_seeds(7,11)
index_non_positive_7_11=[k for k in range(len(Data_7_11)) if k not in index_positive_7_11]

index_positive_8_12= [25]
Data_8_12 = load_seeds(8,12)
index_non_positive_8_12=[k for k in range(len(Data_8_12)) if k not in index_positive_8_12]

Data_9_13 = load_seeds(9,13)
index_non_positive_9_13=[k for k in range(len(Data_9_13))]

def is_iso_nonpositive(K):
    is_isom= False
    for k in index_non_positive_9_13:
        L = Data_9_13[k]
        if sc.are_isom(K,L):
            is_isom = True
            break
    return is_isom

def has_link_isom_nonpositive(K):
    is_isom=False
    for v in range(K.m):
        link_K_v=sc.Link_of(K,sc.list_2_pow[v])
        if is_iso_nonpositive(link_K_v):
            is_isom=True
            break
    return is_isom


list_positive = []
Data = load_seeds(10,14)
counter = 0

for l in tqdm.tqdm(range(len(Data))):
    K = Data[l]
    if has_link_isom_nonpositive(K):
        continue
    seed_is_positive = False
    (m,n) = (K.m,K.n)
    orientation = np.zeros(len(K.facets_bin))
    orientation[0]=1
    while np.any(orientation == 0):
        nonoriented = np.where(orientation==0)[0]
        oriented = np.where(orientation!=0)[0]
        next_guys = []
        for i in nonoriented:
            facet_bin_1 = K.facets_bin[i]
            for j in oriented:
                facet_bin_2 = K.facets_bin[j]
                vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                if len(vertices)==2:
                    vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                    v1=vertices[0]
                    facet1 = sc.binary_to_face_0(facet_bin_1,m)
                    v2=vertices[1]
                    facet2 = sc.binary_to_face_0(facet_bin_2,m)
                    if v1 not in sc.binary_to_face_0(facet_bin_1,m):
                        v1, v2 = v2, v1
                    orientation[i]=(-1)**(facet2.index(v2)+facet1.index(v1)+1)*orientation[j]
                    break 
    for mod_2_char_map_bin in tqdm.tqdm(sc.IDCM_Garrison_Scott(K)):
        mod_2_char_map = sc.char_funct_bin_to_numpy(mod_2_char_map_bin,K.m-K.n)
        seed_is_positive |= is_positive(K,mod_2_char_map,orientation)
        if seed_is_positive:
            break
    if seed_is_positive:
        list_positive.append(l)
print("list of positive: ",list_positive)


100%|██████████| 39/39 [00:05<00:00,  6.84it/s]

list of positive:  []




4-8 -> 11: [2,3,4,5,6,7,8,9,11,12,14]

5-9 -> 16: [6, 7, 8, 10,  11, 18, 19, 20, 28, 47, 53, 76, 92, 95, 97, 108]

6-10 -> 18: [5,6,11,55,72,126,127,188,271,293,309,326,330,352,392,550,583,616]

7-11 -> 9: [12, 162, 210, 247, 355, 359, 556, 1117, 1155]

8-12 -> 1: [25]

Then -> []


In [ ]:
import tqdm

index_positive_4_8=[2, 3, 4, 5, 6, 11, 12, 14]
Data_4_8 = load_seeds(4,8)
index_non_positive_4_8=[k for k in range(len(Data_4_8)) if k not in index_positive_4_8]

index_positive_5_9=[6, 7, 11, 18, 28, 47, 53, 76, 95, 97, 108]
Data_5_9 = load_seeds(5,9)
index_non_positive_5_9=[k for k in range(len(Data_5_9)) if k not in index_positive_5_9]

index_positive_6_10=[5, 11, 72, 127, 188, 293, 309, 326, 392, 583, 616]
Data_6_10 = load_seeds(6,10)
index_non_positive_6_10=[k for k in range(len(Data_6_10)) if k not in index_positive_6_10]


index_positive_7_11= [210, 355, 359, 1155]
Data_7_11 = load_seeds(7,11)
index_non_positive_7_11=[k for k in range(len(Data_7_11)) if k not in index_positive_7_11]

index_positive_8_12= []
Data_8_12 = load_seeds(8,12)
index_non_positive_8_12=[k for k in range(len(Data_8_12)) if k not in index_positive_8_12]

Data_9_13 = load_seeds(9,13)
index_non_positive_9_13=[k for k in range(len(Data_9_13))]

def is_iso_nonpositive(K):
    is_isom= False
    for k in index_non_positive_7_11:
        L = Data_7_11[k]
        if sc.are_isom(K,L):
            is_isom = True
            break
    return is_isom

def has_link_isom_nonpositive(K):
    is_isom=False
    for v in range(K.m):
        link_K_v=sc.Link_of(K,sc.list_2_pow[v])
        if is_iso_nonpositive(link_K_v):
            is_isom=True
            break
    return is_isom


list_positive = []
Data = load_seeds(9,13)
counter = 0

for l in tqdm.tqdm(range(len(Data))):
    K = Data[l]
    if has_link_isom_nonpositive(K):
        continue
    seed_is_positive = False
    (m,n) = (K.m,K.n)
    orientation = np.zeros(len(K.facets_bin))
    orientation[0]=1
    while np.any(orientation == 0):
        nonoriented = np.where(orientation==0)[0]
        oriented = np.where(orientation!=0)[0]
        next_guys = []
        for i in nonoriented:
            facet_bin_1 = K.facets_bin[i]
            for j in oriented:
                facet_bin_2 = K.facets_bin[j]
                vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                if len(vertices)==2:
                    vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                    v1=vertices[0]
                    facet1 = sc.binary_to_face_0(facet_bin_1,m)
                    v2=vertices[1]
                    facet2 = sc.binary_to_face_0(facet_bin_2,m)
                    if v1 not in sc.binary_to_face_0(facet_bin_1,m):
                        v1, v2 = v2, v1
                    orientation[i]=(-1)**(facet2.index(v2)+facet1.index(v1)+1)*orientation[j]
                    break 
    for mod_2_char_map_bin in tqdm.tqdm(sc.IDCM_Garrison_Scott(K)):
        mod_2_char_map = sc.char_funct_bin_to_numpy(mod_2_char_map_bin,K.m-K.n)
        seed_is_positive |= is_positive_and_pm1(K,mod_2_char_map,orientation)
        if seed_is_positive:
            break
    if seed_is_positive:
        list_positive.append(l)
print("list of positive and +-1: ",list_positive)

100%|██████████| 776/776 [00:50<00:00, 15.27it/s]

list of positive and +-1:  []


Results:

- 2, 6 : [0] 1
- 3, 7 : [0, 1, 2, 3] 4
- 4, 8 : [2, 3, 4, 5, 6, 11, 12, 14] 8
- 5, 9 : [6, 7, 11, 18, 28, 47, 53, 76, 95, 97, 108] 11
- 6, 10 : [5, 11, 72, 127, 188, 293, 309, 326, 392, 583, 616] 11
- 7, 11 : [210, 355, 359, 1155] 4
- Then : []


In [8]:
import tqdm
import json

def is_TU(matrix):
    n, m = matrix.shape
    # if not (np.all(np.sum(matrix,axis=0)%2==0) and np.all(np.sum(matrix,axis=1)%2==0) and np.sum(matrix)%4==2):
    #     return False
    if n==m and round(np.linalg.det(matrix)) not in [1,0,-1]:
        # print("hello", np.linalg.det(matrix))
        return False
    for size in range(1, n+1):
        for row_comb in combinations(range(n), size):
            for col_comb in combinations(range(m), size):
                submatrix = matrix[np.ix_(row_comb, col_comb)]
                det = round(np.linalg.det(submatrix))
                if det not in [-1, 0, 1]:
                    # print(det)
                    return False
    return True

def enumerate_cone_tu_pm1(K:sc.PureSimplicialComplex,mod_2_IDCM,orientation,data):
    (m,n) = (K.m,K.n)
    mod_2_char_map = np.zeros((n,m),)
    mod_2_char_map[:,:n] = np.eye(n)
    mod_2_char_map[:,n:] = mod_2_IDCM[:,:n].T
    indexes_support = []
    facets_list = [np.array(sc.binary_to_face_0(facet_bin,m)) for facet_bin in K.facets_bin]
    N = len(facets_list)
    for j in range(n,m):
        for i in range(n):
            if mod_2_char_map[i,j]==1:
                indexes_support.append((i,j))
    for x in tqdm.tqdm(range(2**len(indexes_support))):
        char_map_candidate=mod_2_char_map.copy()
        for k in range(len(indexes_support)):
            if (x>>k)&1 == 1:
                char_map_candidate[indexes_support[k][0],indexes_support[k][1]] = -1
        candidate=True
        for k in range(N):
            if np.abs(np.linalg.det(char_map_candidate[:,facets_list[k]])- orientation[k])> 0.0000001:
                candidate = False
                break
            elif not is_TU(char_map_candidate[:,facets_list[k]]):
                candidate = False
                break
        if not candidate:
            continue
        if is_TU(char_map_candidate):
            continue
        data.append([[int(x) for x in row] for row in char_map_candidate])



list_positive = [2, 3, 4, 5, 6, 11, 12, 14]
Data = load_seeds(4,8)
counter = 0
Dico={}
for l in tqdm.tqdm(list_positive):
    K = Data[l]
    Dico[l]=[]
    seed_is_positive = False
    (m,n) = (K.m,K.n)
    orientation = np.zeros(len(K.facets_bin))
    orientation[0]=1
    while np.any(orientation == 0):
        nonoriented = np.where(orientation==0)[0]
        oriented = np.where(orientation!=0)[0]
        next_guys = []
        for i in nonoriented:
            facet_bin_1 = K.facets_bin[i]
            for j in oriented:
                facet_bin_2 = K.facets_bin[j]
                vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                if len(vertices)==2:
                    vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
                    v1=vertices[0]
                    facet1 = sc.binary_to_face_0(facet_bin_1,m)
                    v2=vertices[1]
                    facet2 = sc.binary_to_face_0(facet_bin_2,m)
                    if v1 not in sc.binary_to_face_0(facet_bin_1,m):
                        v1, v2 = v2, v1
                    orientation[i]=(-1)**(facet2.index(v2)+facet1.index(v1)+1)*orientation[j]
                    break
    for mod_2_char_map_bin in sc.IDCM_Garrison_Scott(K):
        mod_2_char_map = sc.char_funct_bin_to_numpy(mod_2_char_map_bin,K.m-K.n)
        enumerate_cone_tu_pm1(K,mod_2_char_map,orientation,Dico[l])


f=open('./cone_tu_data/4_8','w')
json.dump(Dico,f,indent=4)
f.close()

100%|██████████| 8/8 [04:45<00:00, 35.67s/it]


not tu Cone-tu char maps:


1) Trouver les eventails cone-TU
2) trouver le inégalités pour les b_i
3) trouver les Hilbert

In [9]:
def check_Todd_genus(K,char_map):
    (m,n) = (K.m,K.n)
    facets_list = [np.array(sc.binary_to_face_0(facet_bin,m)) for facet_bin in K.facets_bin]
    N = len(facets_list)
    x = np.dot(char_map[:,facets_list[0]],np.ones(n))
    had_Todd_genus_1 = True
    for k in range(1,N):
        if np.all(np.dot(np.linalg.inv(char_map[:,facets_list[k]]),x)>=0):
            had_Todd_genus_1 = False
            break
    if not had_Todd_genus_1:
        return False
    return True